In [1]:
!pip install pandas==1.0.3
!pip install tensorflow==2.2.0-rc2
!pip install tensorflow-addons


     |████████████████████████████████| 10.0 MB 2.6 MB/s 
ERROR: hypertools 0.6.2 has requirement scikit-learn<0.22,>=0.19.1, but you'll have scikit-learn 0.22.2.post1 which is incompatible.
ERROR: datalab 1.1.5 has requirement pandas-profiling==1.4.0, but you'll have pandas-profiling 2.4.0 which is incompatible.
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.1
    Uninstalling pandas-1.0.1:
      Successfully uninstalled pandas-1.0.1
     |████████████████████████████████| 516.2 MB 1.7 kB/s 
     |████████████████████████████████| 454 kB 58.1 MB/s 
     |████████████████████████████████| 3.0 MB 46.4 MB/s 
     |████████████████████████████████| 777 kB 34.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling ga

In [2]:
import tensorflow as tf
print(tf.__version__)
import copy
import math
import os
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import tensorflow.keras.layers as L
from tensorflow.keras import Model
from sklearn.metrics import f1_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ModelCheckpoint
import math
import gc
import os
from sklearn.metrics import recall_score, f1_score, precision_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from sklearn.model_selection import KFold, GroupKFold
from tensorflow.python.keras.engine import training
import collections
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns

LEN_ = 4000
n_epoch = 350
batch_size = 64
folds = 5
learning_rate = 0.001
SEED_ = 987654321
n_classes = 11

lr_schedule = LearningRateScheduler(
    PolynomialDecay(
      initial_learning_rate = learning_rate,
      decay_steps = n_epoch,
      power = 0.5,
      end_learning_rate = 0.0
    )
)
class FBetaScore(tf.keras.metrics.Metric):
    """Computes F-Beta score.
    It is the weighted harmonic mean of precision
    and recall. Output range is [0, 1]. Works for
    both multi-class and multi-label classification.
    F-Beta = (1 + beta^2) * (prec * recall) / ((beta^2 * prec) + recall)
    Args:
        num_classes: Number of unique classes in the dataset.
        average: Type of averaging to be performed on data.
            Acceptable values are `None`, `micro`, `macro` and
            `weighted`. Default value is None.
        beta: Determines the weight of precision and recall
            in harmonic mean. Determines the weight given to the
            precision and recall. Default value is 1.
        threshold: Elements of `y_pred` greater than threshold are
            converted to be 1, and the rest 0. If threshold is
            None, the argmax is converted to 1, and the rest 0.
    Returns:
        F-Beta Score: float
    Raises:
        ValueError: If the `average` has values other than
        [None, micro, macro, weighted].
        ValueError: If the `beta` value is less than or equal
        to 0.
    `average` parameter behavior:
        None: Scores for each class are returned
        micro: True positivies, false positives and
            false negatives are computed globally.
        macro: True positivies, false positives and
            false negatives are computed for each class
            and their unweighted mean is returned.
        weighted: Metrics are computed for each class
            and returns the mean weighted by the
            number of true instances in each class.
    """

    def __init__(self,
                 num_classes,
                 shape = [LEN_, n_classes],
                 average=None,
                 beta=1.0,
                 threshold=None,
                 name='fbeta_score',
                 dtype=tf.float32):
        super(FBetaScore, self).__init__(name=name)

        if average not in (None, 'micro', 'macro', 'weighted'):
            raise ValueError("Unknown average type. Acceptable values "
                             "are: [None, micro, macro, weighted]")

        if not isinstance(beta, float):
            raise TypeError("The value of beta should be a python float")

        if beta <= 0.0:
            raise ValueError("beta value should be greater than zero")

        if threshold is not None:
            if not isinstance(threshold, float):
                raise TypeError(
                    "The value of threshold should be a python float")
            if threshold > 1.0 or threshold <= 0.0:
                raise ValueError("threshold should be between 0 and 1")

        self.num_classes = num_classes
        self.average = average
        self.beta = beta
        self.threshold = threshold
        self.axis = None
        self.init_shape = shape
        
        if self.average != 'micro':
            self.axis = 0
            # self.init_shape = [shape]

        def _zero_wt_init(name):
            return self.add_weight(
                name,
                shape=self.init_shape,
                initializer='zeros',
                dtype=self.dtype)

        self.true_positives = _zero_wt_init('true_positives')
        self.false_positives = _zero_wt_init('false_positives')
        self.false_negatives = _zero_wt_init('false_negatives')
        self.weights_intermediate = _zero_wt_init('weights_intermediate')

    # TODO: Add sample_weight support, currently it is
    # ignored during calculations.
    def update_state(self, y_true, y_pred, sample_weight=None):
        if self.threshold is None:
            threshold = tf.reduce_max(y_pred, axis=-1, keepdims=True)
            # make sure [0, 0, 0] doesn't become [1, 1, 1]
            # Use abs(x) > eps, instead of x != 0 to check for zero
            y_pred = tf.logical_and(y_pred >= threshold,
                                    tf.abs(y_pred) > 1e-12)
        else:
            y_pred = y_pred > self.threshold

        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)

        def _count_non_zero(val):
            non_zeros = tf.math.count_nonzero(val, axis=self.axis)
            return tf.cast(non_zeros, self.dtype)

        self.true_positives.assign_add(_count_non_zero(y_pred * y_true))
        self.false_positives.assign_add(_count_non_zero(y_pred * (y_true - 1)))
        self.false_negatives.assign_add(_count_non_zero((y_pred - 1) * y_true))
        self.weights_intermediate.assign_add(_count_non_zero(y_true))

    def result(self):
        precision = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_positives)
        recall = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_negatives)

        mul_value = precision * recall
        add_value = (tf.math.square(self.beta) * precision) + recall
        mean = (tf.math.divide_no_nan(mul_value, add_value))
        f1_score = mean * (1 + tf.math.square(self.beta))

        if self.average == 'weighted':
            weights = tf.math.divide_no_nan(
                self.weights_intermediate,
                tf.reduce_sum(self.weights_intermediate))
            f1_score = tf.reduce_sum(f1_score * weights)

        elif self.average is not None:  # [micro, macro]
            f1_score = tf.reduce_mean(f1_score)

        return f1_score

    def get_config(self):
        """Returns the serializable config of the metric."""

        config = {
            "num_classes": self.num_classes,
            "average": self.average,
            "beta": self.beta,
        }

        if self.threshold is not None:
            config["threshold"] = self.threshold

        base_config = super(FBetaScore, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def reset_states(self):
        self.true_positives.assign(tf.zeros(self.init_shape, self.dtype))
        self.false_positives.assign(tf.zeros(self.init_shape, self.dtype))
        self.false_negatives.assign(tf.zeros(self.init_shape, self.dtype))
        self.weights_intermediate.assign(tf.zeros(self.init_shape, self.dtype))
class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

get_custom_objects().update({'Mish': Mish(mish)})

class macroF1(Callback):
    def __init__(self, model, inputs, targets, step = 10):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)
        self.step = step

    def on_train_begin(self, logs={}):    
            # Initialization code    
            self.epochs = 0    

    def on_epoch_end(self, epoch, logs):
        self.epochs += 1   
        if self.epochs % self.step == 0:  
          pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
          f1_val = f1_score(self.targets, pred, average="macro")
          print("Step: {}  --  val_f1_macro_score: {}".format(self.epochs, f1_val))
def Classifier(shape_ = (LEN_, 3)):
    
    def bilstm(x, n_units):
        x = L.Bidirectional(L.GRU(n_units, return_sequences=True))(x)
        return(x)

    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [kernel_size**i for i in range(n)]
        x = layers.Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = layers.Multiply()([tanh_out, sigm_out])
            x = layers.Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = layers.Add()([res_x, x])
            
        return res_x
    
    inp = layers.Input(shape = (shape_))
    
    x = wave_block(inp, 16, 3, 6)
    x = wave_block(x, 32, 3, 4)
    x = wave_block(x, 32, 3, 4)
    x = wave_block(x, 64, 3, 2)
    x = wave_block(x, 64, 3, 2)
    x = wave_block(x, 128, 3, 1)

    x = bilstm(x, 64)
    x = L.Dropout(0.1, noise_shape = (None, 1, 128))(x)

    x = L.TimeDistributed(L.Dense(128, activation = 'Mish'), name = 'first_distributed')(x)
    x = L.Dropout(0.1, noise_shape = (None, 1, 128))(x)
    
    x = L.TimeDistributed(L.Dense(64, activation = 'Mish'), name = 'second_distributed')(x)
    out = L.TimeDistributed(L.Dense(n_classes, activation = 'softmax'), name = 'output')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    
    model.compile(Adam(learning_rate = learning_rate), loss = [tfa.losses.SigmoidFocalCrossEntropy()], 
          metrics=[FBetaScore(n_classes, average = "macro")])
    return model

def load_trained_model(weights_path, shape_):
    modelf = Classifier(shape_)
    modelf.load_weights(weights_path)
    
    return(modelf)


2.2.0-rc2


# IMPORT

In [3]:
path = '../input/kalman-add/'

group = np.load(path + 'group.npy', allow_pickle = True)
test_signal = np.load(path + 'test_signal.npy', allow_pickle = True)
train_signal = np.load(path + 'train_signal.npy', allow_pickle = True)
train_target = np.load(path + 'train_target.npy', allow_pickle = True)
model = Classifier((None, train_signal.shape[-1]))

pred_ = np.load('../input/wavenet-kalman-clean-2/pred.npy', allow_pickle = True)
pred_o = np.load('../input/wavenet-without-out/pred.npy', allow_pickle = True)

# WAVENET

In [4]:
gc.collect()
kf = GroupKFold(n_splits = folds)
pred_nn = np.zeros((train_signal.shape[0], train_signal.shape[1], 11))

for fold_n, (train_index, valid_index) in enumerate(kf.split(train_signal, train_target, group)):
    print(f'BEGIN FOLD:     {fold_n} -------\n\n\n')

    X_train, X_valid = train_signal[train_index,:,:], train_signal[valid_index,:,:]
    y_train, y_valid = train_target[train_index,:,:], train_target[valid_index,:,:]

    model = load_trained_model("../input/wavenet-kalman-clean-2/model" + str(fold_n) + ".hdf5", (None, train_signal.shape[-1]))
    
    pred_nn[valid_index, :, :] = model.predict(X_valid)

del X_train, X_valid, y_train, y_valid, model
gc.collect()

pred_nn = pred_nn.reshape((-1, n_classes))

BEGIN FOLD:     0 -------



BEGIN FOLD:     1 -------



BEGIN FOLD:     2 -------



BEGIN FOLD:     3 -------



BEGIN FOLD:     4 -------





In [5]:
np.save('pred_wavenet_oof.npy', pred_nn, allow_pickle = True)
np.save('pred_wavenet.npy', pred_, allow_pickle = True)

# WAVENET NO OUT

In [6]:
out_ = np.arange(910, 956)
idx = np.arange(train_signal.shape[0])
bool_mask = np.isin(idx, out_)
train_signal = train_signal[~bool_mask]
group = group[~bool_mask]
train_target = train_target[~bool_mask]


In [7]:
gc.collect()
kf = GroupKFold(n_splits = folds)
pred_nn = np.zeros((train_signal.shape[0], train_signal.shape[1], 11))

for fold_n, (train_index, valid_index) in enumerate(kf.split(train_signal, train_target, group)):
    print(f'BEGIN FOLD:     {fold_n} -------\n\n\n')

    X_train, X_valid = train_signal[train_index,:,:], train_signal[valid_index,:,:]
    y_train, y_valid = train_target[train_index,:,:], train_target[valid_index,:,:]

    model = load_trained_model("../input/wavenet-without-out/model" + str(fold_n) + ".hdf5", (None, train_signal.shape[-1]))
    
    pred_nn[valid_index, :, :] = model.predict(X_valid)

del X_train, X_valid, y_train, y_valid, train_signal, train_target, model
gc.collect()

pred_nn = pred_nn.reshape((-1, n_classes))

BEGIN FOLD:     0 -------



BEGIN FOLD:     1 -------



BEGIN FOLD:     2 -------



BEGIN FOLD:     3 -------



BEGIN FOLD:     4 -------





In [8]:
np.save('pred_wavenet_no_out_oof.npy', pred_nn, allow_pickle = True)
np.save('pred_wavenet_no_out.npy', pred_o, allow_pickle = True)

# BILSTM

In [9]:
for i in range(folds):
    path = f'../input/bilstm-final-fold{i}/'
    if i == 0:
        pred = np.load(path + f'pred{i}.npy', allow_pickle = True)/folds
        pred_oof = np.load(path + f'predoof{i}.npy', allow_pickle = True)
    else:
        pred += np.load(path + f'pred{i}.npy', allow_pickle = True)/folds
        pred_oof += np.load(path + f'predoof{i}.npy', allow_pickle = True)

pred_oof = pred_oof.reshape((-1, n_classes))
pred = pred.reshape((-1, n_classes))

np.save('pred_bilstm_oof.npy', pred_oof, allow_pickle = True)
np.save('pred_bilstm.npy', pred, allow_pickle = True)

# BILSTM NO-OUT

In [10]:
for i in range(folds):
    path = f'../input/bilstm-noout-fold{i}/'
    if i == 0:
        pred = np.load(path + f'pred{i}.npy', allow_pickle = True)/folds
        pred_oof = np.load(path + f'predoof{i}.npy', allow_pickle = True)
    else:
        pred += np.load(path + f'pred{i}.npy', allow_pickle = True)/folds
        pred_oof += np.load(path + f'predoof{i}.npy', allow_pickle = True)

pred_oof = pred_oof.reshape((-1, n_classes))
pred = pred.reshape((-1, n_classes))

np.save('pred_bilstm_noout_oof.npy', pred_oof, allow_pickle = True)
np.save('pred_bilstm_noout.npy', pred, allow_pickle = True)

In [11]:
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})

sub['open_channels'] = np.argmax(pred, axis = 1).reshape(-1)
sub.to_csv('submission.csv', index=False)